In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

In [ ]:
df_damaged = pd.read_csv('../data/processed/damagedProperty.csv')
df_eviction = pd.read_csv('../data/processed/Eviction.csv')
df_fire = pd.read_csv('../data/processed/fireIncidents.csv')
df_graffiti = pd.read_csv('../data/processed/graffiti.csv')
df_homeless = pd.read_csv('../data/processed/homelessConcerns.csv')
df_illegalp = pd.read_csv('../data/processed/illegalPostings.csv')
df_incidents = pd.read_csv('../data/processed/incidents.csv')
df_noise = pd.read_csv('../data/processed/noiseReports.csv')

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (12,13,14,15,18,19,20,45,46,55,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Use same name for columns

In [ ]:
df_damaged.info()

In [ ]:
df_eviction.info()

In [ ]:
df_fire.info()

In [ ]:
df_eviction.rename(columns={'SupervisorDistrict':'Supervisor District',
                           'EvictionID':'ID'}, inplace=True)
df_fire.rename(columns={'Arrival DtTm':'Date',
                       'Incident Number':'ID'}, inplace=True)
df_damaged.rename(columns={'Opened':'Date',
                          'CaseID':'ID'},  inplace=True)
df_graffiti.rename(columns={'Opened':'Date',
                           'CaseID':'ID'},  inplace=True)
df_homeless.rename(columns={'Opened':'Date',
                           'CaseID':'ID'},  inplace=True)
df_illegalp.rename(columns={'Opened':'Date',
                           'CaseID':'ID'},  inplace=True)
df_noise.rename(columns={'Opened':'Date',
                        'CaseID':'ID'},  inplace=True)

In [ ]:
df_damaged['Damaged'] = 1 
df_eviction['Eviction'] = 1
df_fire['Fire Incident'] = 1
df_graffiti['Graffiti'] = 1
df_homeless['Homeless Concern'] = 1
df_illegalp['Illegal Postings'] = 1
df_noise['Noise Complaints'] = 1

df_damaged['Category'] = "damaged property"
df_eviction['Category'] = "eviction"
df_fire['Category'] = "fire incident"
df_graffiti['Category'] = "graffiti"
df_homeless['Category'] = "homeless concern"
df_illegalp['Category'] = "illegal postings"
df_noise['Category'] = "noise complaint"

df = []

Merge datasets

In [ ]:
df = df_damaged[['ID','Date','Year','Month','Day','Supervisor District', 'Category','Damaged']].append(df_eviction[['ID','Date','Year','Month','Day','Category','Supervisor District','Eviction']], sort=False)
df.head()

In [ ]:
df.tail()

In [ ]:
df.fillna(0, inplace=True)
df.head()

In [ ]:
df = df.append(df_fire[['ID','Date','Year','Month','Day', 'Category','Supervisor District','Fire Incident']], sort=False)
df.head()

In [ ]:
df = df.append(df_graffiti[['ID','Date','Year','Month','Day', 'Category','Supervisor District','Graffiti']], sort=False)
df.head()

In [ ]:
df = df.append(df_homeless[['ID','Date','Year','Month','Day', 'Category','Supervisor District','Homeless Concern']], sort=False)
df.head()

In [ ]:
df = df.append(df_illegalp[['ID','Date','Year','Month','Day', 'Category','Supervisor District','Illegal Postings']], sort=False)
df.head()

In [ ]:
df = df.append(df_noise[['ID','Date','Year','Month','Day', 'Category','Supervisor District','Noise Complaints']], sort=False)
df.head()

In [ ]:
df.fillna(0, inplace=True)
df.head()

In [ ]:
df.tail()

## Encode

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['CategoryLE'] = le.fit_transform(df["Category"].astype(str))

## Linear Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
# See the relation between off and def columns
sns.jointplot(x=df['CategoryLE'],y=df['Supervisor District'],
              joint_kws={'alpha':0.4, 's':5, 'color':'orange'},
              marginal_kws={'color':'blue'})


In [ ]:
# Split the data between 80% train and 20% test
labels = df['Supervisor District']
features = df[['Damaged','Eviction','Fire Incident','Graffiti',
                        'Homeless Concern','Illegal Postings', 'Noise Complaints']]
train_features, test_features, train_labels, test_labels = train_test_split(features,
                                                                            labels, 
                                                                            test_size=0.20)

# Model the data
model = LinearRegression()
model.fit(train_features, train_labels)
model.score(test_features, test_labels)

In [ ]:
pred_labels = model.predict(test_features) 
sns.regplot(x = test_labels, 
            y = pred_labels, 
            data = model,
            line_kws={"color":"orange","alpha":0.5,"lw":2}
           )

## Other test

In [ ]:
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from sklearn.preprocessing import StandardScaler

# Rescaling the attributes so they have a comparable scale
rfm_df = df[['CategoryLE','Supervisor District']]

# # Instantiate
scaler = StandardScaler()

# fit_transform
rfm_df_scaled = scaler.fit_transform(rfm_df)

rfm_df_scaled = pd.DataFrame(rfm_df_scaled)
rfm_df_scaled.columns = ['rating','ratingCount']
rfm_df_scaled.head()

mergings = linkage(rfm_df_scaled, method="single", metric='euclidean')
dendrogram(mergings)
plt.show()

## Visualization

In [ ]:
# # Load an example dataset with long-form data

# Plot the responses for different events and regions
sns.lineplot(x="Date", y="CategoryLE",
             hue="Supervisor District",
             data=df)

In [ ]:
dots = sns.load_dataset("df")

# Define a palette to ensure that colors will be
# shared across the facets
# palette = dict(zip(dots..unique(),
#                    sns.color_palette("rocket_r", 6)))

# Plot the lines on two facets
sns.relplot(x="Date", y="CategoryLE",
            hue="Supervisor District", size="choice", col="align",
            size_order=["T1", "T2"], palette=palette,
            height=5, aspect=.75, facet_kws=dict(sharex=False),
            kind="line", legend="full", data=dots)